↑プロジェクトトークンを挿入したら、実行を忘れずに!


# WiDSTokyoIBM-2020-20210-Analysis

こちらのQiita記事のものです：

このnotebookはIBMCloud上のWatson Studioのnotebookで動くように書かれていますが、他の環境でも動作すると思いますが、一部Watson Studio特有のCodeがあります。<br>
**`IBM Cloud Watson Studioのみ`** と書いてある場合はWatson Studio特有のCodeです。<br>
逆に他の環境だと必須のCodeはコメントアウトして **`IBM Cloud Watson Studio以外`** と書いてありますので、他の環境はコメントアウト部分を必要に応じて実行してください。


### **`IBM Cloud Watson Studioのみ`**
Watson Studio上のnotebookからIBM Cloud Object Storage(ICOS)へのFileの読み書き - project-libを使う -の手順が前準備として必要です。

まずは1,2を実施お願いします。

- 1.[ Projectを開く](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#1-project%E3%82%92%E9%96%8B%E3%81%8F)
- 2.[ 前準備アクセス・トークンの作成](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#2-%E5%89%8D%E6%BA%96%E5%82%99%E3%82%A2%E3%82%AF%E3%82%BB%E3%82%B9%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E4%BD%9C%E6%88%90)

その後、このnotebookに

- [5: プロジェクト・トークンの挿入](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#5-%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88%E3%83%88%E3%83%BC%E3%82%AF%E3%83%B3%E3%81%AE%E6%8C%BF%E5%85%A5)を実施してください。


一番上に挿入されたセルの実行は忘れずにお願いします。

## 1.  Watson Libraryのインポート
https://cloud.ibm.com/apidocs/natural-language-understanding?code=python


In [ ]:
!pip install --upgrade ibm-watson

 # 2. データファイルの読み込み
 
 DATA列に分析したいデータを入れたエクセルファイルまたはCSVファイルを準備します。<br>
 DATA列のみ使いますので、他の列は何が入っていてもOKです。
 
 サンプル表：
 
 |  KEYID | DATA |
| :--- | :--- |
|  A0001  |  とてもよい感じで感動しました  |
|  B0001 |  見にくい資料があったのでフォントを大きくして欲しい  |

 ## `IBM Cloud Watson Studioのみ`

詳細は[こちら](https://qiita.com/nishikyon/items/912ef69736fcee4c0793)を参照してください

1. 右上の01/00アイコンをクリックし、対象ファイルをドラッグ&ドロップして、ファイルをアップロード
2. 下のセルをクリック
2. アップロードしたファイルの下にある「コードに挿入」メニューをクリックし、`pandas DataFrame`をクリック
3. 下のセルに挿入されたコードの最後の二行が`df_data_0`で始まっていない場合は、`df_data_n`(nは数字) を`df_data_0`に置き換える
3. 下のセルを実行

修正前コードの例:
```
df_data_1 = pd.read_excel(body.read())
df_data_1.head()
```


修正後コード(必ず`df_data_0`で始まるようにする):
```
df_data_0 = pd.read_excel(body.read())
df_data_0.head()
```

In [ ]:
#　Watson Studio利用の場合のみ実行
# pandas DataFrameメニューをクリックする前に、このセルをクリック！！！！！


## `IBM Cloud Watson Studio以外`
 
 data_pathに読み込ませたいEXCELファイル　または　CSVファイルをパス付きで指定後、実行してください。
 
 
 例：
 ```
  data_path= "/Users/nishito/polls_opentext.xlsx"
 ```

In [ ]:
#　ファイルパスを指定して読み込み (Local jupyter実行用)　する場合のみ実行

# import os
# import pandas as pd
# data_path="<data_pathに読み込ませたいEXCELファイル　または　CSVファイルをパス付きで指定>"

# ext_str=os.path.splitext(data_path)
# if ext_str[1].lower() == ".xlsx":
#     df_data_0 = pd.read_excel(data_path)
# elif ext_str[1].lower() == ".csv":
#      df_data_0 = pd.read_csv(data_path)
# else:
#     print("Filepath Error!")
# df_data_0.head()

## 3. いろいろ前処理

各セルの実行をします。
何をやっているかはコメントを参考にしてください。

In [ ]:
# データのコピー
df_poll=df_data_0.copy()

In [ ]:
#データの整形　　refineTextt列に整形したデータを入れる
import numpy as np

def refine_text(df_data):
    
    # データの整形
    df_data['refineText'] = df_data['DATA']

    # NaN削除
    df_data['refineText'] = df_data['refineText'].replace(np.nan, '', regex=True)

    # 改行削除
    df_data['refineText'] = df_data.apply(lambda x: x['refineText'].replace('\n', ' '), axis=1)
    df_data['refineText'] = df_data.apply(lambda x: x['refineText'].strip(), axis=1)
    
    return

refine_text(df_poll)


## 4. Natural Language Understanding: apikey, urlのセット

Natural Language Understandingのapikey, urlを以下のセルでにセットしてください。


```py
apikey="<apileyの文字列>"
url="<urlの文字列>"
```
例：
```py
apikey="u3RTGnji90okmbhuYTGVr-ZAqwSCXCDErfvbgt"
url="https://api.jp-tok.natural-language-understanding.watson.cloud.ibm.com/instances/123456njuhfgNHY-1q2w3e4r5t6yNHY"
```

In [ ]:
apikey="<apileyの文字列>"
url="<urlの文字列>"

## 5.  Natural Language Understanding: NaturalLanguageUnderstandingV1インスタンス作成

参考: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/watson-nlp.html?context=cpdaas&audience=wdp

In [ ]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
def get_NLU(apikey, url):
    authenticator = IAMAuthenticator(apikey)
    nlu = NaturalLanguageUnderstandingV1(
        version='2022-04-07',
        authenticator=authenticator
    )

    nlu.set_service_url(url)
    return nlu

nlu=get_NLU(apikey, url)

## 6. Natural Language UnderstandingでrefineText列(DATA列を整形した列)の分析を実施


In [ ]:
import json
import datetime
from ibm_watson.natural_language_understanding_v1  import Features,  KeywordsOptions,  SentimentOptions

keyword_num = 10
keyword_relevance = 0.3

features = Features(keywords=KeywordsOptions(limit=3), sentiment=SentimentOptions())
df_keywords = pd.DataFrame(columns=['keyword1', 'keyword2', 'keyword3', 'keyword4', 'keyword5', 'keyword6',  'keyword7', 'keyword8',  'keyword9', 'keyword10'])
df_sentiment = pd.DataFrame(columns=['sentiment_label', 'sentiment_score'])
df_keywords_wordcloud  = pd.DataFrame(columns=['keyword', 'sentiment_label', 'sentiment_score'])


for text, i in zip(df_poll['refineText'], df_poll.index):
    if i % 10 == 0:
        print('{0}: Now processing line {1}/{2}.'.format(str(datetime.datetime.now()), i, len(df_poll)))
 
    if not text:
        data_row = ['', '', '']
        df_keywords.loc[i] = data_row
        data_row = ['', np.nan]
        df_sentiment.loc[i] = data_row
        continue

    try:
        enriched_json = nlu.analyze(text=text, features=features, language='ja').get_result()  # NLU　呼び出し
    except ApiException as ex:
        print('{}:text: {} -- Method failed with status code {}: {}'.format(i, text, str(ex.code), ex.message))
        data_row = ['ERR', str(ex.code), ex.message]
        df_keywords.loc[i] = data_row
        data_row = ['ERR', np.nan]
        df_sentiment.loc[i] = data_row
        continue

    # Iterate and get KEYWORDS with a confidence of over keyword_relevance
    if 'keywords' in enriched_json:
        keywords = []
        keynum = keyword_num
        for kw in enriched_json["keywords"]:
            # print(kw["text"])
            keynum -= 1
            if (float(kw["relevance"]) >= keyword_relevance):
                keywords.append(kw["text"])
            else:
                keywords.append('')
        for key_i in range(keynum):
            keywords.append('')
        df_keywords.loc[i] = keywords
    else:
        data_row = ['', '', '','', '', '','', '', '','']
        df_keywords.loc[i] = data_row

    if 'sentiment' in enriched_json:
        sentiment = []
        sentiment.append(enriched_json["sentiment"]["document"]["label"])
        sentiment.append(enriched_json["sentiment"]["document"]["score"])
        df_sentiment.loc[i] = sentiment
    else:
        data_row = ['N/A', np.nan]
        df_sentiment.loc[i] = data_row

df_enriched_poll = pd.concat([df_poll, df_sentiment, df_keywords, ], axis=1)
print("Finished!")


### 7. WatsonStudioのプロジェクトのデータ資産に保存

#### **`IBM Cloud Watson Studioのみ`**
中身の確認はプロジェクトのデータ資産から！

In [ ]:
saved_file_name = "analysis.csv"
project.save_data(saved_file_name,df_enriched_poll.to_csv(index=False),overwrite=True)

#### **`IBM Cloud Watson Studio以外`**

Local Diskに保存します

In [ ]:
# saved_file_name = "analysis.csv"
# # ローカルに保存 
# df_enriched_poll.to_csv(saved_file_name, index=False)
# # 中身の確認
# !head {saved_file_name}

## 8. Word Cloud分析

### 8.1 wordcloudの準備

日本語フォントは　https://moji.or.jp/ipafont/ipaex00401/　よりダウンロードします。
https://moji.or.jp/ipafont/ipaex00401/　のIPAフォントライセンスV1.0を読んで内容に同意の上、実行してください。

In [ ]:
# fontの準備
!wget https://moji.or.jp/wp-content/ipafont/IPAexfont/ipaexg00401.zip
!unzip ipaexg00401.zip
!ls -la  ipaexg00401

In [ ]:
#wordcloud ライブラリの導入
!pip install wordcloud

In [ ]:
# wordcloudの初期化
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#font_pathは環境で変更してください
wordcloud = WordCloud(font_path='ipaexg00401/ipaexg.ttf',
                     width=1600,  height=800,  background_color="white" )

### 8.2 KeywordでWordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

words=''
for key_i in range(keyword_num):
    colname='keyword{0}'.format(key_i+1)
    tmp_words = ' '.join(df_enriched_poll[colname])
    words = words + tmp_words
    
wordcloud.generate(words)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### 8.3 センチメント数のグラフ

In [ ]:
df = df_enriched_poll['sentiment_label'].value_counts()
df.plot(kind='bar')

### 8.4 Positive KeywordでWordcloud

In [ ]:
df = df_enriched_poll.query('sentiment_label == "positive"')

words=''
for key_i in range(keyword_num):
    colname='keyword{0}'.format(key_i+1)
    tmp_words = ' '.join(df[colname])
    words = words + tmp_words
        
wordcloud.generate(words)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### 8.5 Negaitive KeywordでWordcloud

In [ ]:
df = df_enriched_poll.query('sentiment_label == "negative"')

words=''
for key_i in range(keyword_num):
    colname='keyword{0}'.format(key_i+1)
    tmp_words = ' '.join(df[colname])
    words = words + tmp_words
    
wordcloud.generate(words)
plt.figure(figsize=(16,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### 8.6 positiveコメントベスト20
あくまでもコメントのpositiveスコアが高いだけで、主流のコメントではないことにご注意ください。

In [ ]:
df = df_enriched_poll.query('sentiment_label == "positive"')
df.sort_values('sentiment_score', ascending=False).loc[:,['refineText','sentiment_score']].head(20)

###  8.7 negativeコメント ワースト10
あくまでもコメントのnegativeスコアが高い(というかsentiment_scoreが低い)だけで、主流のコメントではないことにご注意ください。

In [ ]:
df = df_enriched_poll.query('sentiment_label == "negative"')
df.sort_values('sentiment_score').loc[:,['refineText','sentiment_score']].head(10)

以上です。
保存したセンチメント情報とキーワード情報のついたファイルもながめてみてください。
ファイル名は`analysis.csv`です。